In [3]:
pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd

In [5]:
url = 'DadosParaDesafio-OtimizaçãoTransporte.xlsx'

data_pedidos = pd.read_excel(url, sheet_name = "Pedido")
data_produtos = pd.read_excel(url, sheet_name = "Produto")
data_veiculos = pd.read_excel(url, sheet_name = "Veículos")
data_destinatarios = pd.read_excel(url, sheet_name = "Destinatário")


features = ['Código', 'Produto', 'Destinatário', 'Quantidade']
pedidos = data_pedidos[features]

features = ['Código', 'Disponível', 'Peso (kg)', 'Volume (m³)', 'Altura', 'Largura', 'Comprimento']
produtos = data_produtos[features]

features = ['Placa', 'Latitude', 'Longitude', 'Lotação (kg)', 'Cubagem (m³)', 'Altura', 'Largura', 'Comprimento', 'Custo / Km', 'Tempo de trabalho (horas)']
veiculos = data_veiculos[features]

features = ['Código', 'Latitude', 'Longitude', '*início recebimento', '*Fim recebimento']
destinatarios = data_destinatarios[features]

In [6]:
agrupado = pedidos.groupby('Destinatário')

volume_destinatario = []
peso_destinatario = []
lista_destinatarios = []

for d, p in agrupado:
       
    volume_total = 0
    peso_total = 0
    
    lista_destinatarios.append(d)
    
    for PRD in p.Produto:

        soma = float(produtos.loc[produtos['Código'] == PRD]['Volume (m³)']) * float(sum((p.loc[p['Produto'] == PRD]['Quantidade'])))
        volume_total = volume_total + soma
        
        soma = float(produtos.loc[produtos['Código'] == PRD]['Peso (kg)']) * float(sum((p.loc[p['Produto'] == PRD]['Quantidade'])))
        peso_total = peso_total + soma
    
    volume_destinatario.append(volume_total)
    peso_destinatario.append(peso_total)
    
print('Foi')

Foi


In [7]:
peso_produtos = {produto: peso for produto, peso in zip(lista_destinatarios, peso_destinatario)}
volume_produtos = {produto: volume for produto, volume in zip(lista_destinatarios, volume_destinatario)}

capacidade_veiculos = {veiculo: capacidade for veiculo, capacidade in zip(veiculos['Placa'], veiculos['Lotação (kg)'])}
area_veiculos = {veiculo: area for veiculo, area in zip(veiculos['Placa'], veiculos['Cubagem (m³)'])}
custo_veiculos = {veiculo: custo for veiculo, custo in zip(veiculos['Placa'], veiculos['Custo / Km'])}

In [8]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model()

# Uma variavel, se o produto está alocado no veiculo em questão
x = model.addVars(lista_destinatarios, veiculos['Placa'], vtype=GRB.BINARY, name='x')
y = model.addVars(veiculos['Placa'], vtype=GRB.BINARY, name='y')

# Função Objetivo: Maximizar o número de itens transportados
model.setObjective(
    gp.quicksum(y[j] * custo_veiculos[j] for j in veiculos['Placa']),
    sense=GRB.MINIMIZE)

# Restrição 1: Cada objeto deve ser alocado apenas uma vez
c1 = model.addConstrs(
    gp.quicksum(x[i, j] for j in veiculos['Placa']) == 1 for i in lista_destinatarios)

# Restrição 2: O somatório dos pesos deve ser menor que a capacidade do veiculo
c2 = model.addConstrs(
    gp.quicksum(x[i, j] * peso_produtos[i]  for i in lista_destinatarios) <= capacidade_veiculos[j] for j in veiculos['Placa'])

# Restrição 3: O somatório dos volumes deve ser menor que a area do veículo
c3 = model.addConstrs(
    gp.quicksum(x[i, j] * volume_produtos[i]  for i in lista_destinatarios) <= area_veiculos[j] for j in veiculos['Placa'])

# Restrição 4
c4 = model.addConstrs(
    x[i, j] <= y[j] for i in lista_destinatarios for j in veiculos['Placa'])

Restricted license - for non-production use only - expires 2023-10-25


In [10]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads


GurobiError: ignored

In [ ]:
# Valor objetivo
print("Número de Produtos:", model.objVal)

lista_de_destinatarios = []

cont = 1
numero_de_destinos = 0
for j in veiculos['Placa']:
    print('', cont,': ', j)
    lista = []
    cont = cont + 1
    for i in lista_destinatarios:        
        if x[i, j].x == 1:
            print('', i)
            lista.append(i)
            numero_de_destinos = numero_de_destinos + 1
    lista_de_destinatarios.append(lista)

In [ ]:
lista_soma_peso = []
lista_soma_volume = []
for i in lista_de_destinatarios:
    soma_peso = 0
    soma_volume = 0
    for j in i:
        soma_peso = soma_peso + peso_produtos[j]
        soma_volume = soma_volume + volume_produtos[j]
    lista_soma_peso.append(soma_peso)
    lista_soma_volume.append(soma_volume)

In [ ]:
cont = 0
for i, j in enumerate(veiculos['Placa']):
    cont = cont + 1
    print('', cont,'-', j)
    print('-----------')
    print('Soma do Peso: ', lista_soma_peso[i])
    print('Capacidade do veículo: ', capacidade_veiculos[j])
    print('Soma do Volume: ', lista_soma_volume[i])
    print('Área do veículo: ', area_veiculos[j])
    print()

In [ ]:
for i in veiculos['Placa']:
    if y[i].x == 1:
        print('Foi Usado: ', i)
    else:
        print('----> Não Foi Usado: ', i)

# Determinar a rota

In [ ]:
import math
from itertools import combinations

rotas_finais = []
custo_da_rota = []

for index, rota in enumerate(lista_de_destinatarios):

    data = rota
    
    capitals = []
    
    for i in data:
        capitals.append(i)
    capitals.append('Origem')
        
    coordinates = destinatarios.query('Código in @data')[['Latitude', 'Longitude']]
    coordinates.index = data
    
    origem = pd.DataFrame([[veiculos.iloc[index].Latitude, veiculos.iloc[index].Longitude]], 
                          columns=['Latitude', 'Longitude'], index=['Origem'])
    
    if len(coordinates)>0:
        coordinates = coordinates.append(origem)
    else:
        rotas_finais.append([])
        custo_da_rota.append(0)
        continue
        
    print(coordinates)

    def distance(city1, city2):
        c1 = coordinates.loc[city1]
        c2 = coordinates.loc[city2]
       
        diff = (c1[0]-c2[0], c1[1]-c2[1])
        
        return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

    dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}
    
    print('---------------------------------------------------------------------------------')
    print('--------------------------------------', index+1,'-------------------------------------------')
    print('---------------------------------------------------------------------------------')
    
    import gurobipy as gp
    from gurobipy import GRB

    m = gp.Model()

    vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

    for i, j in vars.keys():
        vars[j, i] = vars[i, j]  

    cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

    def subtourelim(model, where):
        if where == GRB.Callback.MIPSOL:
            vals = model.cbGetSolution(model._vars)
            selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                                 if vals[i, j] > 0.5)
            tour = subtour(selected)
            if len(tour) < len(capitals):
                model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                             <= len(tour)-1)

    def subtour(edges):
        unvisited = capitals[:]
        cycle = capitals[:] 
        while unvisited:  
            thiscycle = []
            neighbors = unvisited
            while neighbors:
                current = neighbors[0]
                thiscycle.append(current)
                unvisited.remove(current)
                neighbors = [j for i, j in edges.select(current, '*')
                             if j in unvisited]
            if len(thiscycle) <= len(cycle):
                cycle = thiscycle 
        return cycle
    
    m._vars = vars
    m.Params.lazyConstraints = 1
    m.optimize(subtourelim)

    vals = m.getAttr('x', vars)
    selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

    tour = subtour(selected)

    assert len(tour) == len(capitals)

    rotas_finais.append(tour)
    custo_da_rota.append(m.objVal)

In [ ]:
for i, rota in enumerate(rotas_finais):
    print('Rota:')
    print(rota)
    print()
    print('Custo da rota: ', custo_da_rota[i])
    print()
    print()
    print('------------------------------------------')